# Mathematical Lifts: От дискретного к непрерывному

## Что такое "математический подъём"?

**Подъём (lift)** — это преобразование дискретной структуры в более богатую математическую среду, сохраняющее существенные свойства.

### Зачем это нужно?

В этом туториале мы научимся:

1. **Измерять расстояние между строками** — расстояние Левенштейна показывает, сколько редактирований нужно, чтобы превратить одну строку в другую

2. **Превращать строки в числовые векторы** — каждая строка становится точкой в ℝⁿ с координатами-признаками (длина, частоты символов, биграммы)

3. **Работать с вероятностями** — вместо отдельных строк описываем распределения и считаем энтропию

4. **Находить стационарные распределения** — какие строки система "предпочитает" в долгосрочной перспективе

5. **Описывать систему категорным языком** — строки как объекты, правила как морфизмы

### Сквозной пример

Будем использовать **унарную арифметику** из первого туториала:
- Символы: `0` (единица), `|` (пара)
- Правила: `00→|` (объединение), `||→0` (свёртка)

Это минимальная система, но она демонстрирует все концепции!

In [1]:
# Настройка окружения
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

from rewriting import String, Rule, RewritingEngine

# Наша сквозная система: унарная арифметика
rules = [
    Rule(String.from_str("00"), String.from_str("|")),   # 00 → |
    Rule(String.from_str("||"), String.from_str("0")),   # || → 0
]
engine = RewritingEngine(rules)

# Начальная строка
initial = String.from_str("0000")

print("="*60)
print("СКВОЗНОЙ ПРИМЕР: Унарная арифметика")
print("="*60)
print()
print("Правила:")
print("  R1: 00 → |  (две единицы → пара)")
print("  R2: || → 0  (две пары → единица)")
print()
print(f"Начальная строка: '{initial}'")
print()

# Найдём все достижимые строки
from collections import deque

queue = deque([initial])
all_strings = {initial}
depth = {str(initial): 0}

while queue:
    current = queue.popleft()
    for new_s, rule, pos in engine.all_applications(current):
        if new_s not in all_strings:
            all_strings.add(new_s)
            depth[str(new_s)] = depth[str(current)] + 1
            queue.append(new_s)

print(f"Достижимые строки ({len(all_strings)}):")
for s in sorted(all_strings, key=lambda x: (len(str(x)), str(x))):
    print(f"  '{s}' (глубина {depth[str(s)]})")

# Сохраним для последующих разделов
STRINGS = list(all_strings)

СКВОЗНОЙ ПРИМЕР: Унарная арифметика

Правила:
  R1: 00 → |  (две единицы → пара)
  R2: || → 0  (две пары → единица)

Начальная строка: '0000'

Достижимые строки (6):
  '0' (глубина 3)
  '||' (глубина 2)
  '00|' (глубина 1)
  '0|0' (глубина 1)
  '|00' (глубина 1)
  '0000' (глубина 0)


---

## 1. Метрическое пространство: Измеряем "расстояние" между строками

### Идея

Хотим отвечать на вопрос: **насколько строки "похожи"?**

Для этого вводим **метрику** — функцию расстояния `d(s₁, s₂)`, которая:
1. `d(s, s) = 0` — расстояние до себя = 0
2. `d(s₁, s₂) = d(s₂, s₁)` — симметрия
3. `d(s₁, s₃) ≤ d(s₁, s₂) + d(s₂, s₃)` — неравенство треугольника

### Расстояние Левенштейна

Самая естественная метрика для строк — **редакционное расстояние**:

> Минимальное число операций (вставка/удаление/замена символа) для превращения одной строки в другую.

**Пример:**
- `d("00", "0|") = 1` (заменить второй `0` на `|`)
- `d("00", "||") = 2` (заменить оба символа)

In [2]:
from math_lifts.geometry import MetricSpace

# Создаём метрическое пространство
metric = MetricSpace()  # По умолчанию использует расстояние Левенштейна

print("="*60)
print("МЕТРИЧЕСКОЕ ПРОСТРАНСТВО")
print("="*60)
print()

# Вычислим расстояния между всеми парами строк
print("Матрица расстояний Левенштейна:")
print()

# Сортируем строки для красивого вывода
sorted_strings = sorted(STRINGS, key=lambda x: (len(str(x)), str(x)))

# Заголовок
header = "       " + "  ".join(f"{str(s):>4}" for s in sorted_strings)
print(header)
print("-" * len(header))

# Строки матрицы
for s1 in sorted_strings:
    row = f"{str(s1):>5}  "
    for s2 in sorted_strings:
        d = metric.distance(s1, s2)
        row += f"{d:>4.0f}  "
    print(row)

print()
print("Интерпретация:")
print("  • d=0: строки идентичны")
print("  • d=1: отличаются на 1 символ")
print("  • d=2: отличаются на 2 символа")
print("  и т.д.")

МЕТРИЧЕСКОЕ ПРОСТРАНСТВО

Матрица расстояний Левенштейна:

          0    ||   00|   0|0   |00  0000
-----------------------------------------
    0     0     2     2     2     2     3  
   ||     2     0     2     2     2     4  
  00|     2     2     0     2     2     2  
  0|0     2     2     2     0     2     2  
  |00     2     2     2     2     0     2  
 0000     3     4     2     2     2     0  

Интерпретация:
  • d=0: строки идентичны
  • d=1: отличаются на 1 символ
  • d=2: отличаются на 2 символа
  и т.д.


### Алгоритм Левенштейна (динамическое программирование)

Как вычисляется расстояние? Строим таблицу размера `(n+1) × (m+1)`:

```
       ""  0   0   |
   ""   0   1   2   3     ← стоимость вставки
    |   1   1   2   2     ← стоимость замены или совпадения
    |   2   2   2   3
```

**Алгоритм:**
1. `dp[0][j] = j` — создать строку из пустой = j вставок
2. `dp[i][0] = i` — удалить всю строку = i удалений
3. `dp[i][j] = min(dp[i-1][j]+1, dp[i][j-1]+1, dp[i-1][j-1]+cost)`
   - где `cost = 0` если символы равны, иначе `1`

In [3]:
# Пошаговая демонстрация алгоритма Левенштейна

def levenshtein_with_trace(s1: str, s2: str):
    """Вычисляет расстояние Левенштейна с визуализацией DP-таблицы."""
    n, m = len(s1), len(s2)
    
    # Создаём DP таблицу
    dp = [[0] * (m + 1) for _ in range(n + 1)]
    
    # Инициализация: превращение пустой строки
    for i in range(n + 1):
        dp[i][0] = i  # удаление i символов
    for j in range(m + 1):
        dp[0][j] = j  # вставка j символов
    
    # Заполнение: выбираем минимум из трёх операций
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]  # символы совпали — бесплатно
            else:
                dp[i][j] = 1 + min(
                    dp[i-1][j],    # удаление
                    dp[i][j-1],    # вставка
                    dp[i-1][j-1]   # замена
                )
    
    return dp

# Пример: расстояние от "||" до "00|"
s1, s2 = "||", "00|"

print(f"Вычисляем d('{s1}', '{s2}')")
print()

dp = levenshtein_with_trace(s1, s2)

# Визуализация таблицы
print("DP-таблица:")
print()
header = "       " + "  ".join(f"{c:>2}" for c in ["ε"] + list(s2))
print(header)
print("-" * len(header))

for i, row in enumerate(dp):
    prefix = s1[i-1] if i > 0 else "ε"
    print(f"  {prefix:>2}   " + "  ".join(f"{v:>2}" for v in row))

print()
print(f"Ответ: d('{s1}', '{s2}') = {dp[-1][-1]}")
print()
print("Путь (обратный ход):")
print(f"  '{s1}' → замена '|'→'0' → '0|' → вставка '0' → '00|'")

Вычисляем d('||', '00|')

DP-таблица:

        ε   0   0   |
---------------------
   ε    0   1   2   3
   |    1   1   2   2
   |    2   2   2   2

Ответ: d('||', '00|') = 2

Путь (обратный ход):
  '||' → замена '|'→'0' → '0|' → вставка '0' → '00|'


---

## 2. Геометрический подъём: Строки как точки в ℝⁿ

### Идея

Превращаем дискретные строки в **векторы чисел** (точки евклидова пространства).

Это позволяет:
- Визуализировать структуру системы
- Применять методы машинного обучения
- Искать кластеры и паттерны

### Как строить вектор из строки?

Используем **признаки (features)**:
1. **Длина строки**
2. **Частоты символов** (сколько `0`, сколько `|`)
3. **N-граммы** (частоты пар `00`, `0|`, `|0`, `||`)

In [4]:
from math_lifts.geometry import GeometricLift, Point

# Создаём геометрический подъём размерности 8
lift = GeometricLift(embedding_dim=8)

print("="*60)
print("ГЕОМЕТРИЧЕСКИЙ ПОДЪЁМ: Строки → Точки в ℝ⁸")
print("="*60)
print()

# Погружаем каждую строку
print("Признаки каждой строки:")
print()

for s in sorted(STRINGS, key=lambda x: (len(str(x)), str(x))):
    s_str = str(s)
    
    # Вычисляем признаки вручную для демонстрации
    length = len(s_str)
    count_0 = s_str.count('0')
    count_pipe = s_str.count('|')
    frac_0 = count_0 / length if length > 0 else 0
    
    # Биграммы
    bigrams = {'00': 0, '0|': 0, '|0': 0, '||': 0}
    for i in range(len(s_str) - 1):
        bg = s_str[i:i+2]
        if bg in bigrams:
            bigrams[bg] += 1
    
    print(f"'{s_str}':")
    print(f"    длина={length}, #0={count_0}, #|={count_pipe}, доля_0={frac_0:.2f}")
    print(f"    биграммы: {bigrams}")
    
    # Получаем точку от GeometricLift
    point = lift.embed_string(s)
    print(f"    → вектор: [{', '.join(f'{x:.2f}' for x in point.coordinates[:4])}...]")
    print()

ГЕОМЕТРИЧЕСКИЙ ПОДЪЁМ: Строки → Точки в ℝ⁸

Признаки каждой строки:

'0':
    длина=1, #0=1, #|=0, доля_0=1.00
    биграммы: {'00': 0, '0|': 0, '|0': 0, '||': 0}
    → вектор: [0.01, 1.00, 0.00, 1.00...]

'||':
    длина=2, #0=0, #|=2, доля_0=0.00
    биграммы: {'00': 0, '0|': 0, '|0': 0, '||': 1}
    → вектор: [0.02, 0.00, 1.00, 0.00...]

'00|':
    длина=3, #0=2, #|=1, доля_0=0.67
    биграммы: {'00': 1, '0|': 1, '|0': 0, '||': 0}
    → вектор: [0.03, 0.67, 0.33, 1.00...]

'0|0':
    длина=3, #0=2, #|=1, доля_0=0.67
    биграммы: {'00': 0, '0|': 1, '|0': 1, '||': 0}
    → вектор: [0.03, 0.67, 0.33, 1.00...]

'|00':
    длина=3, #0=2, #|=1, доля_0=0.67
    биграммы: {'00': 1, '0|': 0, '|0': 1, '||': 0}
    → вектор: [0.03, 0.67, 0.33, 0.00...]

'0000':
    длина=4, #0=4, #|=0, доля_0=1.00
    биграммы: {'00': 3, '0|': 0, '|0': 0, '||': 0}
    → вектор: [0.04, 1.00, 0.00, 1.00...]



### Евклидово расстояние в ℝⁿ

Теперь можем измерять расстояние между точками по формуле:

$$d(\mathbf{p}_1, \mathbf{p}_2) = \sqrt{\sum_{i=1}^{n} (p_{1,i} - p_{2,i})^2}$$

Это **другая** метрика, чем Левенштейн! Она учитывает непрерывную структуру признаков.

In [5]:
import math

# Сравниваем две метрики: Левенштейн vs Евклидова в ℝⁿ

print("="*60)
print("СРАВНЕНИЕ МЕТРИК")
print("="*60)
print()
print("              Левенштейн    Евклидова (ℝ⁸)")
print("-" * 50)

pairs = [
    (String.from_str("0"), String.from_str("||")),
    (String.from_str("00|"), String.from_str("0|0")),
    (String.from_str("0000"), String.from_str("0")),
]

for s1, s2 in pairs:
    # Расстояние Левенштейна
    d_lev = metric.distance(s1, s2)
    
    # Евклидово расстояние
    p1 = lift.embed_string(s1)
    p2 = lift.embed_string(s2)
    d_euc = math.sqrt(sum((a - b)**2 for a, b in zip(p1.coordinates, p2.coordinates)))
    
    print(f"d('{s1}', '{s2}'):     {d_lev:>5.0f}          {d_euc:>6.2f}")

print()
print("Интерпретация:")
print("  • Левенштейн = число редактирований (дискретная)")
print("  • Евклидова = 'геометрическое' расстояние (непрерывная)")
print("  • Разные метрики подходят для разных задач!")

СРАВНЕНИЕ МЕТРИК

              Левенштейн    Евклидова (ℝ⁸)
--------------------------------------------------
d('0', '||'):         2            1.73
d('00|', '0|0'):         2            1.50
d('0000', '0'):         3            2.00

Интерпретация:
  • Левенштейн = число редактирований (дискретная)
  • Евклидова = 'геометрическое' расстояние (непрерывная)
  • Разные метрики подходят для разных задач!


---

## 3. Теоретико-мерный подъём: Вероятности и энтропия

### Идея

Вместо отдельных строк работаем с **распределениями вероятностей**.

**Мера** μ(s) — "вес" или "важность" строки s.
**Вероятность** P(s) — нормированная мера: $\sum_s P(s) = 1$.

### Зачем?

1. Описать **частоту появления** строк при случайном блуждании
2. Найти **стационарное распределение** системы
3. Измерить **неопределённость** через энтропию

In [6]:
from math_lifts.measure import Measure, Probability

print("="*60)
print("МЕРЫ И ВЕРОЯТНОСТИ")
print("="*60)
print()

# Создаём меру "вручную" — присваиваем веса строкам
# Пусть вес = 1/(длина строки) — короткие строки "важнее"

weights = {}
print("Присваиваем веса (мера μ):")
print("  Правило: μ(s) = 1/len(s) — короткие строки важнее")
print()

for s in STRINGS:
    w = 1.0 / len(str(s))
    weights[s] = w
    print(f"  μ('{s}') = 1/{len(str(s))} = {w:.3f}")

measure = Measure(weights)
print()
print(f"Полная мера: μ(X) = Σμ(s) = {measure.total():.3f}")
print()

# Нормализуем в вероятность
prob = measure.normalize()
print("Нормализация в вероятность P = μ/μ(X):")
print()
for s in sorted(STRINGS, key=lambda x: prob(x), reverse=True):
    print(f"  P('{s}') = {prob(s):.3f}  ({prob(s)*100:.1f}%)")

МЕРЫ И ВЕРОЯТНОСТИ

Присваиваем веса (мера μ):
  Правило: μ(s) = 1/len(s) — короткие строки важнее

  μ('0000') = 1/4 = 0.250
  μ('0|0') = 1/3 = 0.333
  μ('||') = 1/2 = 0.500
  μ('|00') = 1/3 = 0.333
  μ('0') = 1/1 = 1.000
  μ('00|') = 1/3 = 0.333

Полная мера: μ(X) = Σμ(s) = 2.750

Нормализация в вероятность P = μ/μ(X):

  P('0') = 0.364  (36.4%)
  P('||') = 0.182  (18.2%)
  P('0|0') = 0.121  (12.1%)
  P('|00') = 0.121  (12.1%)
  P('00|') = 0.121  (12.1%)
  P('0000') = 0.091  (9.1%)


### Энтропия Шеннона

**Энтропия** измеряет неопределённость распределения:

$$H(P) = -\sum_s P(s) \log_2 P(s)$$

**Интерпретация:**
- $H = 0$ — вся вероятность в одной точке (полная определённость)
- $H = \log_2(n)$ — равномерное распределение (максимальная неопределённость)

**Алгоритм:**
1. Для каждой строки вычислить $-p \log_2(p)$
2. Сложить все вклады

In [7]:
import math

print("="*60)
print("ВЫЧИСЛЕНИЕ ЭНТРОПИИ")
print("="*60)
print()

print("Формула: H(P) = -Σ P(s) log₂ P(s)")
print()
print("Пошаговый расчёт:")
print()

total_entropy = 0.0
for s in sorted(STRINGS, key=lambda x: prob(x), reverse=True):
    p = prob(s)
    if p > 0:
        contribution = -p * math.log2(p)
        total_entropy += contribution
        print(f"  s='{s}': P={p:.3f}, -P·log₂(P) = -{p:.3f}·{math.log2(p):.3f} = {contribution:.3f}")

print()
print(f"Энтропия H(P) = {total_entropy:.3f} бит")
print()

# Сравнение с максимальной энтропией
n = len(STRINGS)
max_entropy = math.log2(n)
print(f"Максимальная энтропия (равномерное): H_max = log₂({n}) = {max_entropy:.3f} бит")
print(f"Отношение: H/H_max = {total_entropy/max_entropy:.1%}")
print()
print("Интерпретация:")
print(f"  Наше распределение имеет {100 - total_entropy/max_entropy*100:.0f}% 'структуры'")
print(f"  (отличается от равномерного из-за весов 1/длина)")

ВЫЧИСЛЕНИЕ ЭНТРОПИИ

Формула: H(P) = -Σ P(s) log₂ P(s)

Пошаговый расчёт:

  s='0': P=0.364, -P·log₂(P) = -0.364·-1.459 = 0.531
  s='||': P=0.182, -P·log₂(P) = -0.182·-2.459 = 0.447
  s='0|0': P=0.121, -P·log₂(P) = -0.121·-3.044 = 0.369
  s='|00': P=0.121, -P·log₂(P) = -0.121·-3.044 = 0.369
  s='00|': P=0.121, -P·log₂(P) = -0.121·-3.044 = 0.369
  s='0000': P=0.091, -P·log₂(P) = -0.091·-3.459 = 0.314

Энтропия H(P) = 2.399 бит

Максимальная энтропия (равномерное): H_max = log₂(6) = 2.585 бит
Отношение: H/H_max = 92.8%

Интерпретация:
  Наше распределение имеет 7% 'структуры'
  (отличается от равномерного из-за весов 1/длина)


---

## 4. Эмпирическая мера: Статистика из траекторий

### Идея

Вместо назначения весов "вручную", **наблюдаем** систему и считаем частоты.

**Алгоритм:**
1. Запускаем случайное блуждание из начальной строки
2. На каждом шаге применяем случайное правило
3. Считаем, сколько раз встретилась каждая строка
4. Частота → эмпирическая вероятность

In [8]:
import random
from collections import Counter

print("="*60)
print("ЭМПИРИЧЕСКАЯ МЕРА ИЗ ТРАЕКТОРИЙ")
print("="*60)
print()

# Симулируем 100 случайных блужданий по 20 шагов каждое
num_walks = 100
steps_per_walk = 20

all_visited = []

print(f"Запускаем {num_walks} случайных блужданий по {steps_per_walk} шагов...")
print()

# Пример одной траектории
current = initial
trajectory = [str(current)]

for step in range(steps_per_walk):
    apps = engine.all_applications(current)
    if not apps:
        break
    current, rule, pos = random.choice(apps)
    trajectory.append(str(current))

print("Пример траектории:")
print(f"  {' → '.join(trajectory[:8])}...")
print()

# Собираем статистику со всех траекторий
for _ in range(num_walks):
    current = initial
    all_visited.append(str(current))
    
    for _ in range(steps_per_walk):
        apps = engine.all_applications(current)
        if not apps:
            break
        current, _, _ = random.choice(apps)
        all_visited.append(str(current))

# Считаем частоты
counts = Counter(all_visited)
total = len(all_visited)

print(f"Всего наблюдений: {total}")
print()
print("Эмпирические частоты:")
for s_str, count in sorted(counts.items(), key=lambda x: -x[1]):
    freq = count / total
    print(f"  '{s_str}': {count:>4} раз = {freq:.3f} ({freq*100:.1f}%)")

# Энтропия эмпирического распределения
emp_entropy = -sum((c/total) * math.log2(c/total) for c in counts.values() if c > 0)
print()
print(f"Энтропия эмпирического распределения: {emp_entropy:.3f} бит")

ЭМПИРИЧЕСКАЯ МЕРА ИЗ ТРАЕКТОРИЙ

Запускаем 100 случайных блужданий по 20 шагов...

Пример траектории:
  0000 → 00| → || → 0...

Всего наблюдений: 340

Эмпирические частоты:
  '0000':  100 раз = 0.294 (29.4%)
  '||':   70 раз = 0.206 (20.6%)
  '0':   70 раз = 0.206 (20.6%)
  '|00':   37 раз = 0.109 (10.9%)
  '00|':   33 раз = 0.097 (9.7%)
  '0|0':   30 раз = 0.088 (8.8%)

Энтропия эмпирического распределения: 2.442 бит


---

## 5. Инвариантная мера: Стационарное распределение

### Идея

**Инвариантная мера** — распределение, которое не меняется под действием системы.

Если $P$ инвариантна, то после одного шага переписывания распределение остаётся тем же.

### Алгоритм (Power Iteration)

Это метод поиска собственного вектора марковской матрицы:

1. Начинаем с равномерного распределения
2. Применяем один "шаг" системы: $P' = T \cdot P$
3. Повторяем пока не сойдётся

In [9]:
from collections import defaultdict

print("="*60)
print("ПОИСК ИНВАРИАНТНОЙ МЕРЫ (Power Iteration)")
print("="*60)
print()

strings_list = list(STRINGS)
n = len(strings_list)

# Начальное равномерное распределение
prob_dict = {str(s): 1.0/n for s in strings_list}

print(f"Начальное распределение (равномерное): P(s) = 1/{n} = {1/n:.3f}")
print()

# Итерации
num_iterations = 20
print(f"Запускаем {num_iterations} итераций...")
print()

for iteration in range(num_iterations):
    new_prob = defaultdict(float)
    
    for s in strings_list:
        s_str = str(s)
        
        # Куда можно перейти из s?
        apps = engine.all_applications(s)
        
        if apps:
            # Равномерно распределяем вероятность по образам
            for t, _, _ in apps:
                t_str = str(t)
                new_prob[t_str] += prob_dict[s_str] / len(apps)
        else:
            # Нормальная форма — остаёмся на месте
            new_prob[s_str] += prob_dict[s_str]
    
    # Проверяем сходимость
    diff = sum(abs(new_prob[str(s)] - prob_dict[str(s)]) for s in strings_list)
    
    if iteration < 5 or iteration == num_iterations - 1:
        print(f"Итерация {iteration + 1}: max diff = {diff:.6f}")
    elif iteration == 5:
        print("...")
    
    prob_dict = dict(new_prob)
    
    if diff < 1e-8:
        print(f"Сошлось на итерации {iteration + 1}!")
        break

print()
print("Инвариантное распределение:")
for s_str in sorted(prob_dict.keys(), key=lambda x: -prob_dict[x]):
    if prob_dict[s_str] > 0.001:
        print(f"  P_inv('{s_str}') = {prob_dict[s_str]:.4f}")

print()
print("Интерпретация:")
print("  Нормальные формы '0' и '0|0' имеют ненулевую вероятность,")
print("  потому что траектории 'застревают' в них навсегда.")

ПОИСК ИНВАРИАНТНОЙ МЕРЫ (Power Iteration)

Начальное распределение (равномерное): P(s) = 1/6 = 0.167

Запускаем 20 итераций...

Итерация 1: max diff = 0.777778
Итерация 2: max diff = 0.666667
Итерация 3: max diff = 0.222222
Итерация 4: max diff = 0.000000
Сошлось на итерации 4!

Инвариантное распределение:
  P_inv('0') = 0.7778
  P_inv('0|0') = 0.2222

Интерпретация:
  Нормальные формы '0' и '0|0' имеют ненулевую вероятность,
  потому что траектории 'застревают' в них навсегда.


---

## 6. Категорный взгляд: Объекты и морфизмы

### Идея

**Категория** — максимально абстрактный способ описать структуру:
- **Объекты** — "вещи" (строки, точки, распределения)
- **Морфизмы** — "стрелки" между вещами (правила, преобразования)

### Наша система как категория

| Компонент | Категория переписывания |
|-----------|-------------------------|
| Объекты | Строки: `"0000"`, `"||"`, ... |
| Морфизмы | Применения правил: `"0000" --[00→|]--> "|00"` |
| Композиция | Цепочки преобразований |
| Тождество | `id: s → s` |

In [10]:
print("="*60)
print("КАТЕГОРИЯ ПЕРЕПИСЫВАНИЯ")
print("="*60)
print()

# Строим категорию вручную для понимания
objects = {str(s): s for s in STRINGS}
morphisms = []

print("ОБЪЕКТЫ (строки):")
for name in sorted(objects.keys(), key=lambda x: (len(x), x)):
    print(f"  • {name}")
print()

print("МОРФИЗМЫ (переходы):")
for s in STRINGS:
    apps = engine.all_applications(s)
    for t, rule, pos in apps:
        morphism = f"'{s}' --[{rule.left}→{rule.right} @{pos}]--> '{t}'"
        morphisms.append(morphism)
        print(f"  {morphism}")

print()
print(f"Итого: {len(objects)} объектов, {len(morphisms)} морфизмов")
print()

# Демонстрация композиции
print("КОМПОЗИЦИЯ МОРФИЗМОВ:")
print()
print("  Пример: '0000' → '|00' → '||' → '0'")
print()
print("  f: '0000' → '|00'  (применили 00→| на позиции 0)")
print("  g: '|00'  → '||'   (применили 00→| на позиции 1)")
print("  h: '||'   → '0'    (применили ||→0 на позиции 0)")
print()
print("  Композиция: h ∘ g ∘ f: '0000' → '0'")
print("  (три последовательных применения правил)")

КАТЕГОРИЯ ПЕРЕПИСЫВАНИЯ

ОБЪЕКТЫ (строки):
  • 0
  • ||
  • 00|
  • 0|0
  • |00
  • 0000

МОРФИЗМЫ (переходы):
  '0000' --[00→| @0]--> '|00'
  '0000' --[00→| @1]--> '0|0'
  '0000' --[00→| @2]--> '00|'
  '||' --[||→0 @0]--> '0'
  '|00' --[00→| @1]--> '||'
  '00|' --[00→| @0]--> '||'

Итого: 6 объектов, 6 морфизмов

КОМПОЗИЦИЯ МОРФИЗМОВ:

  Пример: '0000' → '|00' → '||' → '0'

  f: '0000' → '|00'  (применили 00→| на позиции 0)
  g: '|00'  → '||'   (применили 00→| на позиции 1)
  h: '||'   → '0'    (применили ||→0 на позиции 0)

  Композиция: h ∘ g ∘ f: '0000' → '0'
  (три последовательных применения правил)


### Функторы: Связь между уровнями

**Функтор** $F: C \to D$ — "перевод" одной категории в другую с сохранением структуры.

В нашем случае:
- $F_{\text{geom}}$: Строки → Точки (геометрический подъём)
- $F_{\text{prob}}$: Строки → Распределения (вероятностный подъём)

Функтор сохраняет:
- Объекты → Объекты
- Морфизмы → Морфизмы
- Композицию: $F(g \circ f) = F(g) \circ F(f)$

In [11]:
print("="*60)
print("ФУНКТОР: Геометрический подъём")
print("="*60)
print()

print("Функтор F: Rewriting → Geometry")
print()
print("Действие на ОБЪЕКТЫ (строки → точки):")
for s in sorted(STRINGS, key=lambda x: (len(str(x)), str(x)))[:4]:
    point = lift.embed_string(s)
    coords = ', '.join(f'{x:.2f}' for x in point.coordinates[:3])
    print(f"  F('{s}') = Point({coords}...)")

print()
print("Действие на МОРФИЗМЫ (переходы → векторы):")

# Пример морфизма
s1 = String.from_str("0000")
s2 = String.from_str("|00")

p1 = lift.embed_string(s1)
p2 = lift.embed_string(s2)

# Вектор перехода
vector = tuple(p2.coordinates[i] - p1.coordinates[i] for i in range(3))

print(f"  Морфизм: '0000' → '|00'")
print(f"  F(морфизм) = вектор направления:")
print(f"    F('{s1}') = ({p1.coordinates[0]:.2f}, {p1.coordinates[1]:.2f}, ...)")
print(f"    F('{s2}') = ({p2.coordinates[0]:.2f}, {p2.coordinates[1]:.2f}, ...)")
print(f"    Вектор = ({vector[0]:.2f}, {vector[1]:.2f}, {vector[2]:.2f}, ...)")
print()
print("Интерпретация: правило переписывания = направление в пространстве признаков")

ФУНКТОР: Геометрический подъём

Функтор F: Rewriting → Geometry

Действие на ОБЪЕКТЫ (строки → точки):
  F('0') = Point(0.01, 1.00, 0.00...)
  F('||') = Point(0.02, 0.00, 1.00...)
  F('00|') = Point(0.03, 0.67, 0.33...)
  F('0|0') = Point(0.03, 0.67, 0.33...)

Действие на МОРФИЗМЫ (переходы → векторы):
  Морфизм: '0000' → '|00'
  F(морфизм) = вектор направления:
    F('0000') = (0.04, 1.00, ...)
    F('|00') = (0.03, 0.67, ...)
    Вектор = (-0.01, -0.33, 0.33, ...)

Интерпретация: правило переписывания = направление в пространстве признаков


---

## 📊 Итоги: Что мы узнали

### Уровни абстракции

```
    Дискретный мир              Непрерывный мир
    ═════════════               ═══════════════
    
    Строки "00", "|"   ──────►  Точки в ℝⁿ
                        геом.
                        подъём
    
    Правила 00→|       ──────►  Векторные поля
    
    Частоты            ──────►  Вероятностные меры
                        мера
                        
    Графы переходов    ──────►  Категории
                        функтор
```

### Алгоритмы

| Задача | Алгоритм | Сложность |
|--------|----------|-----------|
| Расстояние Левенштейна | Dynamic Programming | O(n·m) |
| Погружение строки | Feature extraction | O(n) |
| Энтропия | Суммирование -p·log(p) | O(|support|) |
| Инвариантная мера | Power Iteration | O(iterations · |V| · |E|) |

### Зачем это нужно?

1. **Визуализация** — геометрический подъём позволяет "увидеть" структуру
2. **Анализ** — меры и энтропия количественно описывают систему
3. **Обобщение** — категорный язык связывает разные представления
4. **ML/AI** — векторные представления для машинного обучения

## 🚀 Следующий шаг

**03_full_integration.ipynb** — Полный пайплайн анализа через `SpaceLanguageOrchestrator`